# IRIS DATA: SIMPLE SCATTER PLOT USING D3 IN JUYPTER NOTEBOOK

## Introduction

This code is based on the blog explanation. The blog explains the various steps involved. Blog Link :


Now let us use the famous Iris data set to represent a scatter plot.
We will use the bl.ocks template from here ,to construct and customize the scatterplot in order to learn more about it .The link to the data set is here
(https://gist.githubusercontent.com/mbostock/3887118/raw/2e68ffbeb23fe4dadd9b0f6bca62e9def6ee9e17/data.tsv) and it is in the tsv format.

Please refer to the blog for the explanation of this note book .

In [55]:
# load the Preliminaries
from IPython.core.display import display, HTML
from string import Template
import pandas as pd
import json, random

In [56]:
# Get the D3 host locally. 
HTML('<script src="./d3.min.js"></script>')

In [47]:
# get the data from the link and parse it to convert it into a datafram-it's in tsv format
filename = 'https://gist.githubusercontent.com/mbostock/3887118/raw/2e68ffbeb23fe4dadd9b0f6bca62e9def6ee9e17/data.tsv'
iris = pd.read_csv(filename,sep="\t")
iris.head()

,sepalLength,sepalWidth,petalLength,petalWidth,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [48]:
# D3 accepts dictionaries , so let's convert data frame into dictionaries.

In [49]:
iris_array_of_dicts = iris.to_dict(orient='records')
iris_array_of_dicts[:5]

[{'petalLength': 1.4,
  'petalWidth': 0.2,
  'sepalLength': 5.1,
  'sepalWidth': 3.5,
  'species': 'setosa'},
 {'petalLength': 1.4,
  'petalWidth': 0.2,
  'sepalLength': 4.9,
  'sepalWidth': 3.0,
  'species': 'setosa'},
 {'petalLength': 1.3,
  'petalWidth': 0.2,
  'sepalLength': 4.7,
  'sepalWidth': 3.2,
  'species': 'setosa'},
 {'petalLength': 1.5,
  'petalWidth': 0.2,
  'sepalLength': 4.6,
  'sepalWidth': 3.1,
  'species': 'setosa'},
 {'petalLength': 1.4,
  'petalWidth': 0.2,
  'sepalLength': 5.0,
  'sepalWidth': 3.6,
  'species': 'setosa'}]

## CSS,HTML,Javascript  based on bl.ocks example

Note that in the below css_text, we have removed the 'body' style reference from the original bl.ocks text. This is to avoid this style changing the rest of the notebook.`m

You can get bl.ocks from here : https://bl.ocks.org/mbostock/3887118

In [51]:
css_text = '''
.axis path,
.axis line {
  fill: none;
  stroke: #000;
  shape-rendering: crispEdges;
}

.dot {
  stroke: #000;
}
'''

Next, let’s copy the java script and make two changes that are :

* The first set of changes is to the width and height of the image
* The second change is simply to reference a different DOM element as the starting point. D3 will look for a specific DOM element to write things to. These block usually reference an external file like csv/tsv. The $ is going to be used in the templet engine to find and replace.


Note : //* — is used as comments , so those lines will not be executed and below that you can see the changes that were made.

In [52]:
js_text_template = Template('''
var margin = {top: 20, right: 20, bottom: 30, left: 40},
// ****    width = 960 - margin.left - margin.right, ****
// ****    height = 500 - margin.top - margin.bottom; ****
    width = 720 - margin.left - margin.right,
    height = 375 - margin.top - margin.bottom;

var x = d3.scale.linear()
    .range([0, width]);

var y = d3.scale.linear()
    .range([height, 0]);

var color = d3.scale.category10();

var xAxis = d3.svg.axis()
    .scale(x)
    .orient("bottom");

var yAxis = d3.svg.axis()
    .scale(y)
    .orient("left");

// **** var svg = d3.select("body").append("svg") ****
var svg = d3.select("#$graphdiv").append("svg")
    .attr("width", width + margin.left + margin.right)
    .attr("height", height + margin.top + margin.bottom)
  .append("g")
    .attr("transform", "translate(" + margin.left + "," + margin.top + ")");

// **** d3.tsv("data.tsv", function(error, data) { ****
// ****  if (error) throw error; ****

var data = $python_data ;

  data.forEach(function(d) {
    d.sepalLength = +d.sepalLength;
    d.sepalWidth = +d.sepalWidth;
  });

  x.domain(d3.extent(data, function(d) { return d.sepalWidth; })).nice();
  y.domain(d3.extent(data, function(d) { return d.sepalLength; })).nice();

  svg.append("g")
      .attr("class", "x axis")
      .attr("transform", "translate(0," + height + ")")
      .call(xAxis)
    .append("text")
      .attr("class", "label")
      .attr("x", width)
      .attr("y", -6)
      .style("text-anchor", "end")
      .text("Sepal Width (cm)");

  svg.append("g")
      .attr("class", "y axis")
      .call(yAxis)
    .append("text")
      .attr("class", "label")
      .attr("transform", "rotate(-90)")
      .attr("y", 6)
      .attr("dy", ".71em")
      .style("text-anchor", "end")
      .text("Sepal Length (cm)")

  svg.selectAll(".dot")
      .data(data)
    .enter().append("circle")
      .attr("class", "dot")
      .attr("r", 3.5)
      .attr("cx", function(d) { return x(d.sepalWidth); })
      .attr("cy", function(d) { return y(d.sepalLength); })
      .style("fill", function(d) { return color(d.species); });

  var legend = svg.selectAll(".legend")
      .data(color.domain())
    .enter().append("g")
      .attr("class", "legend")
      .attr("transform", function(d, i) { return "translate(0," + i * 20 + ")"; });

  legend.append("rect")
      .attr("x", width - 18)
      .attr("width", 18)
      .attr("height", 18)
      .style("fill", color);

  legend.append("text")
      .attr("x", width - 24)
      .attr("y", 9)
      .attr("dy", ".35em")
      .style("text-anchor", "end")
      .text(function(d) { return d; });

// **** }); ****

''')

In [53]:
# Now let’s make a template for the html string
html_template = Template('''
<style> $css_text </style>
<div id="graph-div"></div>
<script> $js_text </script>
''')


You can notice I have put css text for the style which we created earlier.Then we use div instead of a body and use that id to reference later .First we have the template engine for the java script where we give that data and create a connection for graphdive by referencing the id .Now for the next step ,add the css and the java script to the HTML template engine which will combine it to execute the output(the scatter plot)

In [54]:
js_text = js_text_template.substitute({'python_data': json.dumps(iris_array_of_dicts),
                                       'graphdiv': 'graph-div'})
HTML(html_template.substitute({'css_text': css_text, 'js_text': js_text}))